In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from shutil import copy2 as copy
from IPython.display import display, HTML
from math import floor
import json
import re
import os

secrets = json.load(open('secrets.json'))
game_version = secrets['game_version']
os.makedirs(game_version, exist_ok=True)

# Awakening "Player" Classes

In [ ]:
classes_meta = pd.read_csv(f'data/classes_meta.csv', sep='\s*?,\s*?', engine='python').fillna('')
core_maxes = pd.read_csv('data/classes_core.csv')
rank_stats = ['Str', "Mag", 'Skl', 'Spd', 'Def', 'Res']
stats = ['Str', "Mag", 'Skl', 'Spd', 'Def', 'Res']

core_maxes = core_maxes.merge(classes_meta.drop(rank_stats, axis=1), on='Class')
def attack_stat(x):
    if re.search('Reason|Faith|Dark|Staves', x):
        if re.search('Swords|Bows|Lances|Axes|Stones', x):
            return "Dual"
        else: return 'Mag'
    else: return 'Phys'
core_maxes['Atk'] = core_maxes.Wield.apply(lambda x: attack_stat(x))

class_tiers = core_maxes.loc[core_maxes['Rank']=='Advanced', 'Class'].to_frame()
for x in stats:
    class_tiers[x] = pd.cut(core_maxes.loc[core_maxes['Rank']=='Advanced',x], 5, retbins=True, labels=['E', 'D', 'C', 'B', 'A'])[0]
class_data = class_tiers.merge(core_maxes[['Class', 'Atk', 'Wield', 'Rank', 'Tags']], on='Class')

# # Dashes Str/Mag if the unit can't use a weapon of that type, Averages Dual type value to both.
# def _func(x):
#     if x.Atk == 'Mag': x['Str'] = '-'
#     if x.Atk == 'Phys': x['Mag'] = '-'
#     if x.Atk == 'Dual': 
#         x1 = {'E':1, 'D':2, 'C':3, 'B':4, 'A':5}[x['Str']]
#         x2 = {'E':1, 'D':2, 'C':3, 'B':4, 'A':5}[x['Mag']]
#         x['Str'] = {1:'E', 2:'D', 3:'C', 4:'B', 5:'A'}[round((x1+x2-1)/2)]
#         x['Mag'] = {1:'E', 2:'D', 3:'C', 4:'B', 5:'A'}[round((x1+x2-1)/2)]
#     return x
# class_data = class_data.apply(lambda x: _func(x), axis=1).drop(['Atk'], axis=1)

class_tiers['Total'] = class_tiers[stats].apply(lambda x: x.map({'-':0, 'E':1, 'D':2, 'C':3, 'B':4, 'A':5})).sum(axis=1)
class_tiers.sort_values('Total')
# same_stats = class_tiers.groupby(rank_stats)['Class'].apply(','.join).reset_index()
# same_stats = same_stats[same_stats.Class.notnull()]
# same_stats['Count'] = same_stats.Class.str.count(',')+1
# same_stats

In [ ]:
export = class_tiers.copy()
export["Atk"] = export.apply(lambda x: x.Mag if x.Mag != '-' else x.Str, axis=1)
export['HP'] = 20
export_order = ['Class', 'HP', 'Str', 'Mag', 'Skl', 'Spd', 'Def', 'Res', 'Wield', 'Tags', 'Rank']
export = export[export_order].sort_values('Class')
md_table = export.to_markdown(index=0)

export.to_csv(f'{game_version}/classes.csv', index=False)
open(f'{game_version}/Classes.md', 'w').write(md_table)
open(f'{secrets["ObsidianRoot"]}/{game_version}/Classes.md', 'w').write(md_table)
print(md_table)

# Engage Classes

In [2]:
engage_raw = pd.read_csv('data/engage_classes.csv', sep='\s*,\s*', engine='python')
engage_raw['Tags'] = engage_raw['Tags'].fillna('')
stats = ['HP', 'Str', "Mag", 'Dex', 'Spd', 'Def', 'Res']

# Pre rating tuning

for col in ['Def']:
    engage_raw[col] = engage_raw[col].apply(lambda x: x-12 if x > 50 else x)

# Rating generation

class_data = engage_raw.loc[(~engage_raw.Unique), ['Class', 'Weapons']]
for x in stats:
    class_data[x] = pd.cut(engage_raw.loc[(~engage_raw.Unique),x], 5, retbins=True, 
labels=['E', 'D', 'C', 'B', 'A'])[0]
class_data = class_data.merge(engage_raw, on=['Class', 'Weapons'], suffixes=[None, 'Value'])


# Export cleansing and tuning

export = class_data.copy()
export["Tags"] = ((export.Tags + ' ' + export.Weak + ' '+ export.Type).str.strip().str.split(' ')).apply(lambda x: ', '.join(set([y for y in x if y != '—'])))
# export["Tags"] = export.apply(lambda x: x.Tags if x.Bld < 16 else x.Tags+', Built', axis=1)
export["Tags"] = export.Tags.str.replace('QiAdept', 'Qi Adept')
export['Promoted'] = export.Promoted.map({True:"Yes", False:'No'})

for x in stats: export[x] = export[x].astype('str')
grooms = pd.read_csv('data/engage_class_groom.csv', sep='\s*,\s*', engine='python')
for i, x in grooms.iterrows():
    export.loc[export.Class.str.contains(x.Class), stats] = list(x.loc[stats])

base_hp = 35
HP_factor = 2
debug = export[['Class']+stats].copy()
debug.loc[:, stats] = debug.loc[:, stats].apply(lambda x: x.map({'-':0, 'E':1, 'D':2, 'C':3, 'B':4, 
'A':5, 'S':6}))
export['Total'] = export[stats].apply(lambda x: x.map({'-':0, 'E':1, 'D':2, 'C':3, 'B':4, 
'A':5})).sum(axis=1)
export['HP'] = export['HP'].map({'E':base_hp-(2*HP_factor), 'D':base_hp-(HP_factor), 'C':base_hp, 'B':base_hp+HP_factor, 'A':base_hp+(2*HP_factor), 'S':base_hp+(3*HP_factor)})
# export['HP'] = export.HP.astype('int')

export_order = ['Class', 'HP', 'Str', 'Mag', 'Dex', 'Spd', 'Def', 'Res', 'Proficencies', 'Tags', 'Promoted']
export = export.rename({'Weapons':"Proficencies"}, axis=1)
export = export[export_order].sort_values(['Promoted','Class'])
md_table = export.to_markdown(index=0)

same_stats = export.groupby(stats)['Class'].apply(','.join).reset_index()
same_stats = same_stats[export.Class.notnull()]
same_stats['Count'] = same_stats.Class.str.count(',')+1

# ---

# class_data
export.sort_values(['Def']).reset_index(drop=True)
# print((export[stats].agg(pd.value_counts)/52).round(2))
# same_stats
# export#[export.Promoted == 'No']


/var/folders/ks/dksml1ms0rg9dk9rjyqpnfdc0000gn/T/ipykernel_29565/258163681.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  same_stats = same_stats[export.Class.notnull()]


,Class,HP,Str,Mag,Dex,Spd,Def,Res,Proficencies,Tags,Promoted
0,Great Knight,39,B,E,B,E,A,D,Sword[B+] Axe[B+],"Cavalry, Armor",Yes
1,Great Knight,39,B,E,B,E,A,D,Lance[B+] Axe[B+],"Cavalry, Armor",Yes
2,Sword Armor,39,C,E,C,E,A,E,Sword[B],Armor,No
3,Lance Armor,39,C,E,C,E,A,E,Lance[B],Armor,No
4,Great Knight,39,B,E,B,E,A,D,Sword[B+] Lance[B+],"Cavalry, Armor",Yes
5,Axe Armor,39,C,E,C,E,A,E,Axe[B],Armor,No
6,Paladin,37,C,D,C,C,B,C,Lance[A+],Cavalry,Yes
7,Halberdier,37,B,D,B,D,B,D,Lance[S],Backup,Yes
8,Hero,37,B,E,B,C,B,D,Sword[A] Lance[C+],Backup,Yes
9,Hero,37,B,E,B,C,B,D,Sword[A] Axe[C+],Backup,Yes


In [3]:
os.makedirs(f'{secrets["ObsidianRoot"]}/{game_version}', exist_ok=True)
export.to_csv(f'{game_version}/classes_engage.csv', index=False)
md_table = ''
md_table += f'## Base Classes\n\n'
md_table += export.loc[export.Promoted == 'No'].to_markdown(index=0)+'\n\n'
md_table += f'## Promoted Classes\n\n'
md_table += export.loc[export.Promoted == 'Yes'].to_markdown(index=0)+'\n\n'
open(f'{game_version}/Classes.md', 'w').write(md_table)
open(f'{secrets["ObsidianRoot"]}/{game_version}/Classes.md', 'w').write(md_table)
# print(md_table)

7317

In [25]:
debug['Stamina'] = debug['Str']+debug['Def']
debug['Wit'] = debug['Mag']+debug['Res']
debug['Dexterity'] = debug['Dex']+debug['Spd']
debug

,Class,HP,Str,Mag,Dex,Spd,Def,Res,Stamina,Wit,Dexterity
0,Sword Fighter,2,3,1,4,4,3,2,6,3,8
1,Swordmaster,2,3,2,4,6,2,3,5,5,10
2,Hero,4,4,1,4,3,4,2,8,3,7
3,Hero,4,4,1,4,3,4,2,8,3,7
4,Lance Fighter,4,3,1,3,3,3,2,6,3,6
5,Halberdier,4,4,2,4,2,4,2,8,4,6
6,Royal Knight,2,3,3,4,3,3,4,6,7,7
7,Axe Fighter,5,5,1,2,2,2,2,7,3,4
8,Berserker,6,6,1,3,4,1,1,7,2,7
9,Warrior,5,5,1,3,4,3,1,8,2,7
